In [1]:
import pandas as pd
from pandas import Series, DataFrame
import country_converter as coco
import string
import os
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
## Get current working directory
print("Current working directory:", os.getcwd())

## Find path to raw data (for the raw files)
raw_data_path_usgs = os.path.abspath(os.path.join(os.getcwd(), '..', 'raw_data/usgs'))

## Find path to folder for saving cleaned csv
cleaned_data_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'cleaned_data'))


## Find path to folder for inflation
inflation_data_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'inflation'))

Current working directory: /Users/jennagreene/Documents/GitHub/HATCH_data/reading_files


Data sources:
- https://www.usgs.gov/centers/national-minerals-information-center/historical-global-statistics-mineral-and-material
- Data Series 896

In [3]:
def read_usgs_896(tech_name, file_name, sheet_name=0):
    # Read Excel file into DataFrame
    df = pd.read_excel(file_name, sheet_name=sheet_name, header=3,
                       na_values=['XX','--', 'W', '(3/)', '(4/)','-- '])
    
    # Drop rows with irrelevant country names
    drop_idx = []
    for idx in df.index:
        country = df[df.columns[0]].iloc[idx]
        drop_list = ['Limonite', 'Other', 'Puerto Rico', 'United States: Puerto Rico', 'Leeward and Windward Islands ']
        if country in drop_list:
            drop_idx.append(idx)
    df.drop(index=drop_idx, inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    # Slice DataFrame until 'World' row
    for idx in df.index:
        country = df[df.columns[0]].iloc[idx]
        if country == 'World':
            world_idx = idx + 1
    df = df[:world_idx]
    
    # Extract country names and clean them
    countries = []
    for country in df[df.columns[0]]:
        country = country.strip(string.digits)
        country = country.strip()
        countries.append(country)
    df['Country Name'] = countries
    df.drop(columns=df.columns[0], inplace=True)
    
    # Assign ISO2 country codes and spatial scale
    iso2 = []
    spatial_scale = []
    for country in df['Country Name']:
        if country == 'Serbia and Montenegro':
            iso2.append('CS')
            spatial_scale.append('National')
        elif country == 'Czechoslovakia':
            iso2.append('CSK')
            spatial_scale.append('National')
        elif country == 'U.S.S.R.':
            iso2.append('SU')
            spatial_scale.append('National')
        elif country == 'Yugoslavia':
            iso2.append('YU')
            spatial_scale.append('National')
        elif country == 'Netherlands Antilles':
            iso2.append('AN')
            spatial_scale.append('National')
        elif country == 'World':
            iso2.append('World')
            spatial_scale.append('Global')
        else:
            iso2.append(coco.convert(names=country, to='iso2'))
            spatial_scale.append('National')
    df['Country Code'] = iso2
    df['Spatial Scale'] = spatial_scale
    
    # Add metadata columns
    df['Data Source'] = 'USGS'
    df['Unit'] = 'metric tons'
    df['Metric'] = 'Annual production'
    df['Technology Name'] = tech_name
    df['ID'] = df['Technology Name'] + '_' + df['Metric'] + '_' + df['Country Code']
    df.set_index('ID', inplace=True)
    
    # Remove irrelevant columns
    omit = []
    for col in df.columns:
        if len(str(col)) > 4 or col == 'Unit':
            omit.append(col)
    empty_rows = df.drop(columns=omit)
    empty_rows.dropna(how='all', inplace=True)
    
    # Remove rows with no data
    na_idx = []
    for country in df.index:
        if country not in empty_rows.index:
            na_idx.append(country)
    df.drop(na_idx, inplace=True)
    
    # Remove duplicated entries for US/world (data available in another dataset)
    dup_list = ['Primary Aluminum Production_Annual production_US', 
                'Primary Aluminum Production_Annual production_World',
                'Primary Bauxite Production_Annual production_World','Cadmium Refining_Annual production_US',
                'Cadmium Refining_Annual production_World','Lead_Annual production_US',
                'Lead_Annual production_World','Salt Production_Annual production_US',
                'Iron Ore_Annual production_US','Iron Ore_Annual production_World',
                'Raw Steel Production_Annual production_US','Raw Steel Production_Annual production_World']
    for idx in df.index:
        if idx in dup_list:
            df.drop(idx, inplace=True)

    return df

Aluminum

In [4]:
# call the function
aluminum = read_usgs_896('Primary Aluminum Production', (raw_data_path_usgs + '/ds896-aluminum.xlsx'), 'Aluminum, primary production')

# save df to csv
output_file = 'ds896_Aluminum.csv'
output_file_path = os.path.join(cleaned_data_path, output_file)
aluminum.to_csv(output_file_path)

Bauxite

In [5]:
# call the function
bauxite = read_usgs_896('Primary Bauxite Production', (raw_data_path_usgs + '/ds896-aluminum.xlsx'), 'Bauxite')

# Save df to csv
output_file = 'ds896_Bauxite.csv'
output_file_path = os.path.join(cleaned_data_path, output_file)
bauxite.to_csv(output_file_path)

Cadmium

In [6]:
# call function
cadmium = read_usgs_896('Cadmium Refining', (raw_data_path_usgs + '/ds896-cadmium.xlsx'))

# Omit East/West Germany since there's only one year of data for each
#cadmium = cadmium[(cadmium['Country Name']!='Germany: Eastern states') & (cadmium['Country Name']!='Germany: Western states')]

# save df to csv
output_file = 'ds896_CadmiumRefining.csv'
output_file_path = os.path.join(cleaned_data_path, output_file)
cadmium.to_csv(output_file_path)


Copper (Mining and Refining)

In [7]:
# Copper mining
copper_mining = read_usgs_896('Copper|Mining', (raw_data_path_usgs + '/ds896-copper.xlsx'), 'Mine')
copper_mining = copper_mining[copper_mining['    Mine production']=='Mine: Total']
copper_mining.drop(columns='    Mine production', inplace=True)
#save df to csv
output_file = 'ds896_CopperMining.csv'
output_file_path = os.path.join(cleaned_data_path, output_file)
copper_mining.to_csv(output_file_path)

# Copper refining

copper_refining = read_usgs_896('Copper|Refining', (raw_data_path_usgs + '/ds896-copper.xlsx'), 'Refinery')
copper_refining = copper_refining[copper_refining['Refinery production']=='Refinery: Total']
copper_refining.drop(columns='Refinery production', inplace=True)
#save df to csv
output_file = 'ds896_CopperRefining.csv'
output_file_path = os.path.join(cleaned_data_path, output_file)
copper_refining.to_csv(output_file_path)

Iron ore and raw steel

In [8]:
# Iron ore 
iron = read_usgs_896('Iron Ore', (raw_data_path_usgs +'/ds896-iron-steel.xlsx'), 'Iron ore, gross weight')
#iron = iron[(iron['Country Name']!='Germany: Western states')]
# Save file
output_file = 'ds896_IronOre.csv'
output_file_path = os.path.join(cleaned_data_path, output_file)
iron.to_csv(output_file_path)


# Raw Steel
steel = read_usgs_896('Raw Steel Production', (raw_data_path_usgs +'/ds896-iron-steel.xlsx'), 'Raw steel')
#steel = steel[(steel['Country Name']!='Germany: Eastern states') & (steel['Country Name']!='Germany: Western states')]
# Save file
output_file = 'ds896_RawSteelProduction.csv'
output_file_path = os.path.join(cleaned_data_path, output_file)
steel.to_csv(output_file_path)

Lead

In [9]:
# Lead
lead = read_usgs_896('Lead', (raw_data_path_usgs + '/ds896-lead.xlsx'), 'Mine')
# lead = lead[lead['Country Name']!='Germany: Western states']
output_file = 'ds896_Lead.csv'
output_file_path = os.path.join(cleaned_data_path, output_file)
lead.to_csv(output_file_path)

In [10]:
# Read data from the Excel file for salt production
salt = pd.read_excel((raw_data_path_usgs + '/ds896-salt.xlsx'), sheet_name=0, header=3,
                     na_values=['XX','--', 'W', '(3/)', '(4/)','-- '])

# Drop rows corresponding to irrelevant countries
drop_idx = []
for idx in salt.index:
    country = salt[salt.columns[0]].iloc[idx]
    drop_list = ['Limonite', 'Other', 'Puerto Rico', 'United States: Puerto Rico', 'Leeward and Windward Islands ']
    if country in drop_list:
        drop_idx.append(idx)
salt.drop(index=drop_idx, inplace=True)
salt.reset_index(drop=True, inplace=True)

# Slice the DataFrame to include only relevant data
for idx in salt.index:
    country = salt[salt.columns[0]].iloc[idx]
    if country == 'World':
        world_idx = idx
salt = salt[:world_idx]

# Extract country names
countries = []
for country in salt[salt.columns[0]]:
    country = country.strip(string.digits)
    country = country.strip()
    countries.append(country)
salt['Country Name'] = countries

# Drop columns and rows not needed
salt = salt[(salt['Country Name']!='Germany: Eastern states') & (salt['Country Name']!='Germany: Western states')]
salt.drop(columns=[salt.columns[0], 'Salt type'], inplace=True)

# Summarize production by salt type for each country
unique_countries = sorted(list(set(salt['Country Name'])))
salt_type_sum = []
for country in unique_countries:
    dup_df = salt[salt['Country Name']==country].sum()
    dup_df = dup_df[:-1]
    val_list = []
    for x in dup_df:
        val_list.append(x)
    val_list.append(country)
    salt_type_sum.append(val_list)
salt = pd.DataFrame(salt_type_sum, columns=salt.columns)

# Assign ISO2 country codes and spatial scale
iso2 = []
spatial_scale = []
for country in salt['Country Name']:
    if country == 'Serbia and Montenegro':
        iso2.append('CS')
        spatial_scale.append('National')
    elif country == 'Czechoslovakia':
        iso2.append('CSK')
        spatial_scale.append('National')
    elif country == 'U.S.S.R.':
        iso2.append('SU')
        spatial_scale.append('National')
    elif country == 'Yugoslavia':
        iso2.append('YU')
        spatial_scale.append('National')
    elif country == 'World':
        iso2.append('World')
        spatial_scale.append('Global')
    elif country == 'Netherlands Antilles':
        iso2.append('AN')
        spatial_scale.append('National')
    else:
        iso2.append(coco.convert(names=country, to='iso2'))
        spatial_scale.append('National')
salt['Country Code'] = iso2
salt['Spatial Scale'] = spatial_scale

# Add metadata columns
salt['Data Source'] = 'USGS'
salt['Unit'] = 'metric tons'
salt['Metric'] = 'Annual production'
salt['Technology Name'] = 'Salt Production'
salt['ID'] = salt['Technology Name'] + '_' + salt['Metric'] + '_' + salt['Country Code']
salt.set_index('ID', inplace=True)

# Remove duplicated entries for US (data available in another dataset)
salt.drop('Salt Production_Annual production_US', inplace=True)

# Replace zero values with NaN
salt.replace(0, np.nan, inplace=True)

# Define the output file path and save the processed DataFrame to a CSV file
output_file = 'ds896_SaltProduction.csv'
output_file_path = os.path.join(cleaned_data_path, output_file)
salt.to_csv(output_file_path)

Data source:
- https://www.usgs.gov/centers/national-minerals-information-center/historical-statistics-mineral-and-material-commodities
- Data Series 140


In [11]:
target_inflation_file = "/A001RG3A086NBEA.xls"
target_inflation_filepath = inflation_data_path + target_inflation_file

In [12]:
## adjusting for inflation from 1998 USD to 2022
nipa = pd.read_excel(target_inflation_filepath, header=10)

year_list = []
for x in nipa['observation_date']:
    x = int(str(x)[:4])
    year_list.append(x)
nipa['Year'] = year_list
nipa.set_index('Year', drop=True, inplace=True)
nipa.drop(columns='observation_date', inplace=True)
nipa = nipa.transpose()

infl_1998_2022 = float(nipa[2022]/nipa[1998])
infl_1998_2022

1.6912695457930007

In [13]:
def read_usgs_140(tech_name, file_name, sheet=0, header=4,
                  col_names=['Year','Production','Unit value (98$/t)','World production']):
    """
    Reads data from the USGS 140 dataset.

    Parameters:
    - tech_name (str): Name of the technology.
    - file_name (str): Path to the Excel file containing the data.
    - sheet (int): Index or name of the sheet to read.
    - header (int): Row number to use as the column names.
    - col_names (list): List of column names to read from the Excel file.

    Returns:
    - DataFrame: Processed DataFrame containing the data.
    """
    # Read data from the Excel file
    df = pd.read_excel(file_name, sheet_name=sheet, header=header, usecols=col_names, 
                       na_values=['XX','--', 'W', '(3/)', '(4/)','-- '])
    
    # Find the index where the numeric data ends
    end_idx = None
    for idx in df.index:
        year = df['Year'].iloc[idx]
        if type(year) != int:
            end_idx = idx
            break
    
    # Slice the DataFrame to include only the numeric data
    df = df[:end_idx]
    
    # Set 'Year' column as the index
    df.set_index('Year', drop=True, inplace=True)
    
    # Transpose the DataFrame
    df = df.transpose()
    
    # Adjust production values by inflation rate
    for col in df.columns:
        df[col] = df[col] * infl_1998_2022
    
    # Add metadata columns
    df['Technology Name'] = tech_name
    df['Data Source'] = 'USGS'
    df['Country Name'] = ['United States', 'World', 'World']
    df['Country Code'] = ['US', 'World', 'World']
    df['Spatial Scale'] = ['National', 'Global', 'Global']
    df['Unit'] = ['metric tons', '2022 USD/metric ton', 'metric tons']
    df['Metric'] = ['Annual production', 'Price', 'Annual production']
    
    # Create unique identifier for each row
    df['ID'] = df['Technology Name'] + '_' + df['Metric'] + '_' + df['Country Code']
    
    # Set 'ID' column as the index
    df.set_index('ID', drop=True, inplace=True)
    
    # Remove column name
    df.columns.name = None
    
    # Return the processed DataFrame
    return df

Raw Steel

In [14]:
# raw steel
steel_ds140 = read_usgs_140('Raw Steel Production', (raw_data_path_usgs + '/ds140-iron-steel-2019.xlsx'), header=5,
                            sheet='Steel',
                            col_names=['Year','Raw steel production','Unit value (98$/t)','World production'])

# save file


output_file = 'ds140_RawSteelProduction.csv'
output_file_path = os.path.join(cleaned_data_path, output_file)
steel_ds140.to_csv(output_file_path)

Cadmium

In [15]:
# cadmium
cadmium_ds140 = read_usgs_140('Cadmium Refining', (raw_data_path_usgs + '/ds140-cadmium-2019.xlsx'), 
                             col_names=['Year','Production','Unit value (98 $/t)','World production'])

# save file
output_file = 'ds140_CadmiumRefining.csv'
output_file_path = os.path.join(cleaned_data_path, output_file)
cadmium_ds140.to_csv(output_file_path)

Iron Ore

In [16]:
# iron ore
iron_ds140 = read_usgs_140('Iron Ore', (raw_data_path_usgs + '/ds140-iron-ore-2019.xlsx'),
                          col_names=['Year','Production','Unit value (98$/t) ','World production'])

# save file
output_file = 'ds140_IronOre.csv'
output_file_path = os.path.join(cleaned_data_path, output_file)
iron_ds140.to_csv(output_file_path)

Lead

In [17]:
# lead
lead_ds140 = read_usgs_140('Lead', (raw_data_path_usgs + '/ds140-lead-2018.xlsx'),
                          col_names=['Year','Primary production','Unit value (98$/t)','World production'])

# save file
output_file = 'ds140_Lead.csv'
output_file_path = os.path.join(cleaned_data_path, output_file)
lead_ds140.to_csv(output_file_path)

Primary Aluminum

In [18]:
# primary aluminum
aluminum_ds140 = read_usgs_140('Primary Aluminum Production', (raw_data_path_usgs + '/ds140-aluminum-2019.xlsx'),
                          col_names=['Year','Primary production','Unit value (98$/t)','World production'])


# save file
output_file = 'ds140_Aluminum.csv'
output_file_path = os.path.join(cleaned_data_path, output_file)
aluminum_ds140.to_csv(output_file_path)

Graphite

In [19]:
# graphite
graphite = read_usgs_140('Graphite', (raw_data_path_usgs + '/ds140-graphite-2019.xlsx'))


# save file
output_file = 'ds140_Graphite.csv'
output_file_path = os.path.join(cleaned_data_path, output_file)
graphite.to_csv(output_file_path)

Cobalt

In [20]:
# cobalt
cobalt = read_usgs_140('Cobalt', (raw_data_path_usgs + '/ds140-cobalt-2019.xlsx'),
                    col_names=['Year','Primary production','Unit value (98$/t)','World mine production'])

   

# save file
output_file = 'ds140_Cobalt.csv'
output_file_path = os.path.join(cleaned_data_path, output_file)
cobalt.to_csv(output_file_path)             


Lithium

In [21]:
# lithium
lithium = read_usgs_140('Lithium Mine Production', (raw_data_path_usgs + '/ds140-lithium-2019.xlsx'),
                    col_names=['Year','Production','Unit value (98$/t)','World production (gross weight)'])

# save file
output_file = 'ds140_Lithium.csv'
output_file_path = os.path.join(cleaned_data_path, output_file)
lithium.to_csv(output_file_path)           
                    

Rare Earth

In [22]:
# rare earth
rare_earth = read_usgs_140('Rare Earth Mine Production', (raw_data_path_usgs + '/ds140-rare-earths-2019.xlsx'))

# save file
output_file = 'ds140_RareEarth.csv'
output_file_path = os.path.join(cleaned_data_path, output_file)
rare_earth.to_csv(output_file_path)      


Silver

In [23]:
# silver
silver = read_usgs_140('Silver Production', (raw_data_path_usgs + '/ds140-silver-2018.xlsx'),
                    col_names=['Year','Mine production','Unit value (98$/t)','World production'])

# save file
output_file = 'ds140_Silver.csv'
output_file_path = os.path.join(cleaned_data_path, output_file)
silver.to_csv(output_file_path)     


Tin

In [24]:
# tin
tin = read_usgs_140('Tin Production', (raw_data_path_usgs + '/ds140-tin-2019.xlsx'),
                    col_names=['Year','Primary production','Unit value (98$/t)','World production'])
                    
 
# save file
output_file = 'ds140_Tin.csv'
output_file_path = os.path.join(cleaned_data_path, output_file)
tin.to_csv(output_file_path)     
                   

Cement

In [25]:
# Cement
cement = read_usgs_140('Cement', (raw_data_path_usgs + '/ds140-cement-2019.xlsx'))

# save file
output_file = 'ds140_Cement.csv'
output_file_path = os.path.join(cleaned_data_path, output_file)
cement.to_csv(output_file_path)     
            

Primary Copper

In [26]:
# Primary copper
copper = read_usgs_140('Primary Copper', (raw_data_path_usgs + '/ds140-copper-2018.xlsx'), 
             col_names=['Year','Primary production','Unit value (98$/t)','World production'])

  
# save file
output_file = 'ds140_Copper.csv'
output_file_path = os.path.join(cleaned_data_path, output_file)
copper.to_csv(output_file_path)               

Gold

In [27]:
# Gold
gold = read_usgs_140('Gold Production', (raw_data_path_usgs + '/ds140-gold-2018.xlsx'), 
             col_names=['Year','Primary production','Unit value (98$/t)','World production'])

      
# save file
output_file = 'ds140_Gold.csv'
output_file_path = os.path.join(cleaned_data_path, output_file)
gold.to_csv(output_file_path)            

Nickel

In [28]:
# Nickel
nickel = read_usgs_140('Nickel Production', (raw_data_path_usgs + '/ds140-nickel-2018_1.xlsx'), 
             col_names=['Year','Primary production','Unit value (98$/t)','World production'])

      
# save file
output_file = 'ds140_Nickel.csv'
output_file_path = os.path.join(cleaned_data_path, output_file)
nickel.to_csv(output_file_path)           

Sand and Gravel (Construction)

In [29]:
# Sand and gravel (construction)
sand_gravel_construction = read_usgs_140('Sand and Gravel|Construction', 
                                         (raw_data_path_usgs + '/ds140-sand-gravel-construction-2019.xlsx'))

          
# save file
output_file = 'ds140_SandGravelConstruction.csv'
output_file_path = os.path.join(cleaned_data_path, output_file)
sand_gravel_construction.to_csv(output_file_path)                                  

Sand and Gravel Industrial

In [30]:
# Sand and gravel (industrial)
sand_gravel_industrial = read_usgs_140('Sand and Gravel|Industrial', 
                                         (raw_data_path_usgs + '/ds140-sand-industrial-2019.xlsx'))
           
# save file
output_file = 'ds140_SandGravelIndustrial.csv'
output_file_path = os.path.join(cleaned_data_path, output_file)
sand_gravel_industrial.to_csv(output_file_path)                                         

Zinc

In [31]:
# zinc
zinc = read_usgs_140('Zinc Production', (raw_data_path_usgs + '/ds140-zinc-2019.xlsx'),
                    col_names=['Year',' Primary production','Unit value (98$/t)','World production'])
                         
# save file
output_file = 'ds140_Zinc.csv'
output_file_path = os.path.join(cleaned_data_path, output_file)
zinc.to_csv(output_file_path)         


Salt

In [32]:
# salt
salt_ds140 = read_usgs_140('Salt Production', (raw_data_path_usgs +'/ds140-2017-salt.xlsx'),
                    col_names=['Year','Production','Unit value  (98$/t)','World production'])
                                          
# save file
output_file = 'ds140_Salt.csv'
output_file_path = os.path.join(cleaned_data_path, output_file)
salt_ds140.to_csv(output_file_path)         


Bauxite

In [33]:
# bauxite
bauxite_ds140 = read_usgs_140('Primary Bauxite Production', (raw_data_path_usgs + '/ds140-bauxite-alumina-2019.xlsx'),
                          col_names=['Year','Production','Unit value (98$/t)','World production'])
                                                           
# save file
output_file = 'ds140_Bauxite.csv'
output_file_path = os.path.join(cleaned_data_path, output_file)
bauxite_ds140.to_csv(output_file_path)        